# 🚢 Titanic Survival Prediction

**Author:** Piyush Ramteke  
**Program:** CodSoft Data Science Internship  

---

## 1️⃣ Problem Statement

The sinking of the **Titanic** in 1912 is one of the deadliest maritime disasters in history. Out of 2,224 passengers and crew, more than 1,500 lost their lives.

**🎯 Objective:** Build a Machine Learning model to predict whether a passenger **survived or not** based on features like age, gender, ticket class, fare, and family size.

This is a **binary classification** problem:
- **0** → 💀 Did not survive
- **1** → 🏆 Survived

## 2️⃣ Import Libraries & Load Dataset 📦

In [27]:
# ── Import Libraries ─────────────────────────────────────────

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, widgets
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print('✅ All libraries loaded with Interactive capabilities!')

✅ All libraries loaded with Interactive capabilities!


In [28]:
# ── Load Dataset ─────────────────────────────────────────────

df = pd.read_csv('Titanic-Dataset.csv')

print(f'📂 Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns')
df.head()

📂 Dataset Shape: 891 rows × 12 columns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**📋 Column Descriptions:**

| Feature | Description |
|---------|-------------|
| `PassengerId` | Unique ID for each passenger |
| `Survived` | **Target** — 0 = No, 1 = Yes |
| `Pclass` | Ticket class — 1 = 1st, 2 = 2nd, 3 = 3rd |
| `Name` | Passenger name |
| `Sex` | Gender |
| `Age` | Age in years |
| `SibSp` | Number of siblings/spouses aboard |
| `Parch` | Number of parents/children aboard |
| `Ticket` | Ticket number |
| `Fare` | Ticket fare |
| `Cabin` | Cabin number |
| `Embarked` | Port of embarkation — C = Cherbourg, Q = Queenstown, S = Southampton |

---

## 3️⃣ Exploratory Data Analysis (Interactive) 📊

In [29]:
# ── 3.1 Dataset Info ─────────────────────────────────────────

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [30]:
# ── 3.2 Missing Values Summary ──────────────────────────────

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage (%)': missing_pct
}).sort_values('Missing Count', ascending=False)

print('🔍 Missing Values Summary:')
print('=' * 40)
missing_df[missing_df['Missing Count'] > 0]

🔍 Missing Values Summary:


,Missing Count,Percentage (%)
Cabin,687,77.10
Age,177,19.87
Embarked,2,0.22


**🕵️ Observations:**
- **Cabin** — 77% missing → too many gaps, we'll drop it 🗑️
- **Age** — 19.9% missing → we'll fill with the median age 🎂
- **Embarked** — only 2 missing → we'll fill with the most common port ⚓

In [31]:
# ── 3.3 Interactive Survival Distribution ───────────────────

surv_counts = df['Survived'].value_counts().reset_index()
surv_counts.columns = ['Survived', 'Count']
surv_counts['Label'] = surv_counts['Survived'].map({0: 'Did Not Survive', 1: 'Survived'})

# Creative Interactive Pie Chart
fig = px.pie(surv_counts, values='Count', names='Label', 
             color='Label', 
             color_discrete_map={'Did Not Survive':'#EF553B', 'Survived':'#00CC96'},
             title='📊 Survival Distribution (Interactive)',
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [32]:
# ── 3.4 Interactive: Survival by Feature ─────────────────────

@interact(Feature=['Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch'])
def plot_survival_by_feature(Feature):
    fig = px.histogram(df, x=Feature, color='Survived', 
                       barmode='group',
                       color_discrete_map={0: '#EF553B', 1: '#00CC96'},
                       title=f'Survival Count by {Feature}',
                       text_auto=True)
    fig.update_layout(bargap=0.2)
    fig.show()

interactive(children=(Dropdown(description='Feature', options=('Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch'),…

In [33]:
# ── 3.5 Interactive Age Distribution ────────────────────────

fig = px.histogram(df, x='Age', color='Survived', 
                   nbins=30, 
                   color_discrete_map={0: '#EF553B', 1: '#00CC96'},
                   title='🎂 Age Distribution by Survival Status',
                   marginal='box',
                   opacity=0.7)
fig.update_layout(barmode='overlay')
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.show()

---

## 4️⃣ Data Preprocessing 🛠️

In [34]:
# ── 4.1 Handle Missing Values ──────────────────────────────

# Drop Cabin (too many nan)
df.drop(columns=['Cabin'], inplace=True)

# Fill Age with Median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Fill Embarked with Mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

print('✅ Missing values handled!')
df.isnull().sum()

✅ Missing values handled!


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [35]:
# ── 4.2 Feature Engineering ────────────────────────────────

# Create FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch']

# Drop irrelevant columns
df.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)

print('✅ Feature Engineering complete. New column: FamilySize')

✅ Feature Engineering complete. New column: FamilySize


In [36]:
# ── 4.3 Encode Categorical Features ────────────────────────

le = LabelEncoder()

df['Sex'] = le.fit_transform(df['Sex'])           # female=0, male=1
df['Embarked'] = le.fit_transform(df['Embarked']) # C=0, Q=1, S=2

print('✅ Categorical encoding applied!')
print(f'  Sex      → {dict(zip(["female","male"], [0,1]))}')
print(f'  Embarked → {dict(zip(["C","Q","S"], [0,1,2]))}')
df.head()

✅ Categorical encoding applied!
  Sex      → {'female': 0, 'male': 1}
  Embarked → {'C': 0, 'Q': 1, 'S': 2}


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize
0,0,3,1,22.0,1,0,7.2500,2,1
1,1,1,0,38.0,1,0,71.2833,0,1
2,1,3,0,26.0,0,0,7.9250,2,0
3,1,1,0,35.0,1,0,53.1000,2,1
4,0,3,1,35.0,0,0,8.0500,2,0


---

## 5️⃣ Train/Test Split & Feature Scaling 🧪

In [37]:
# ── 5.1 Separate Features and Target ────────────────────────

X = df.drop('Survived', axis=1)
y = df['Survived']

print(f'Features: {list(X.columns)}')
print(f'Target: Survived')

Features: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize']
Target: Survived


In [38]:
# ── 5.2 Train/Test Split (80-20) ────────────────────────────

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Testing set:  {X_test.shape[0]} samples')

Training set: 712 samples
Testing set:  179 samples


In [39]:
# ── 5.3 Feature Scaling ──────────────────────────────────────

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print('✅ Features Scaled (StandardScaler).')

✅ Features Scaled (StandardScaler).


---

## 6️⃣ Model Training 🤖

In [40]:
# ── 6.1 Logistic Regression ─────────────────────────────────

lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)

print('✅ Logistic Regression Trained.')

✅ Logistic Regression Trained.


In [41]:
# ── 6.2 Random Forest Classifier ────────────────────────────

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)

print('✅ Random Forest Trained.')

✅ Random Forest Trained.


---

## 7️⃣ Model Evaluation 📉

In [42]:
# ── 7.1 Metrics & Interactive Confusion Matrix ──────────────

def evaluate_model(name, y_true, y_pred):
    acc  = accuracy_score(y_true, y_pred)
    print(f'\n🔹 {name} Accuracy: {acc*100:.2f}%')
    print(classification_report(y_true, y_pred))
    return acc

print('🔍 Model Evaluation:')
lr_acc = evaluate_model('Logistic Regression', y_test, lr_pred)
rf_acc = evaluate_model('Random Forest', y_test, rf_pred)

🔍 Model Evaluation:

🔹 Logistic Regression Accuracy: 80.45%
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179


🔹 Random Forest Accuracy: 83.24%
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       105
           1       0.82      0.76      0.79        74

    accuracy                           0.83       179
   macro avg       0.83      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



In [43]:
# ── 7.2 Interactive Confusion Matrix ────────────────────────

@interact(Model=['Logistic Regression', 'Random Forest'])
def plot_conf_matrix(Model):
    if Model == 'Logistic Regression':
        pred = lr_pred
    else:
        pred = rf_pred
    
    cm = confusion_matrix(y_test, pred)
    
    fig = px.imshow(cm, text_auto=True, 
                    labels=dict(x="Predicted", y="Actual", color="Count"),
                    x=['Not Survived', 'Survived'],
                    y=['Not Survived', 'Survived'],
                    title=f'Confusion Matrix - {Model}',
                    color_continuous_scale='Blues')
    fig.show()

interactive(children=(Dropdown(description='Model', options=('Logistic Regression', 'Random Forest'), value='L…

---

## 8️⃣ Conclusion & Interactive Prediction 🔮

The **Random Forest** model typically performs slightly better due to its ability to capture non-linear relationships.

In [44]:
# ── 8.1 Interactive Prediction Widget ───────────────────────

print('🔮 Try predicting survival!')

def predict_survival(Pclass, Sex, Age, SibSp, Parch, Fare, Embarked):
    # Preprocess Input
    sex_enc = 1 if Sex == 'male' else 0
    emb_map = {'C':0, 'Q':1, 'S':2}
    emb_enc = emb_map[Embarked]
    fam_size = SibSp + Parch
    
    # Create DF
    input_data = pd.DataFrame([[Pclass, sex_enc, Age, SibSp, Parch, Fare, emb_enc, fam_size]],
                              columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize'])
    
    # Scale
    input_scaled = scaler.transform(input_data)
    
    # Predict
    prob = rf.predict_proba(input_scaled)[0][1]
    pred = 'Survived 🏆' if prob > 0.5 else 'Did Not Survive 💀'
    
    print(f'\n📢 Model Prediction: {pred}')
    print(f'📊 Survival Probability: {prob*100:.2f}%')

# Create Widget
interact(predict_survival, 
         Pclass=widgets.Dropdown(options=[1, 2, 3], value=3, description='Class:'),
         Sex=widgets.Dropdown(options=['male', 'female'], value='male', description='Gender:'),
         Age=widgets.IntSlider(min=1, max=100, step=1, value=25, description='Age:'),
         SibSp=widgets.IntSlider(min=0, max=8, value=0, description='Siblings:'),
         Parch=widgets.IntSlider(min=0, max=6, value=0, description='Parents:'),
         Fare=widgets.FloatSlider(min=0, max=500, step=10, value=30, description='Fare:'),
         Embarked=widgets.Dropdown(options=['S', 'C', 'Q'], value='S', description='Port:')
);

🔮 Try predicting survival!


interactive(children=(Dropdown(description='Class:', index=2, options=(1, 2, 3), value=3), Dropdown(descriptio…